## Load Libraries

When you click the **Render** button a presentation will be generated that includes both content and the output of embedded code. You can embed code like this:


In [ ]:
#!pip install numpy matplotlib pandas sklearn seaborn calmap squarify plotly
#!pip install calmap

In [ ]:
# Load library for visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import calmap
# Load library for data precessing
import pandas as pd  
import numpy as np

In [ ]:
!pip install wget
!python -m wget -o taipei_sans_tc_beta.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

In [ ]:
# colab顯示繁體中文 問題：matplotlib繪圖，會發生中文無法顯示的問題
# 先下載台北黑體字型
!wget -O taipei_sans_tc_beta.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
# 新增字體
matplotlib.font_manager.fontManager.addfont('taipei_sans_tc_beta.ttf')
# 將 font-family 設為 Taipei Sans TC Beta
# 設定完後，之後的圖表都可以顯示中文了
matplotlib.rc('font', family='Taipei Sans TC Beta')

In [ ]:
# 空氣品質監測日平均值(一般污染物)
# https://data.moenv.gov.tw/dataset/detail/AQX_P_19
# https://drive.google.com/drive/folders/1OrMlB4hP8nnW_0bYwoHRO6DRXWNk1qvy?usp=sharing
!gdown --id '1P3qrYrynZhXDC13dVo5KhDXmld5OGZz1' --output Data/202310.csv
!gdown --id '1P1Kv1ZmPOYyi83DJKUIPoksM31vJJiS5' --output Data/202311.csv
!gdown --id '1P0mojOXgvVbXImnRTPQemm7dCkLDLKAC' --output Data/202312.csv
!gdown --id '1OwAf366l-iItXV4foJemw5QdMuD3JgMc' --output Data/202401.csv

In [ ]:
# 讀檔
df202401 = pd.read_csv('Data/202401.csv') 
df202312 = pd.read_csv('Data/202312.csv') 
df202311 = pd.read_csv('Data/202311.csv') 
df202310 = pd.read_csv('Data/202310.csv') 
df = pd.concat([df202401,df202312,df202311,df202310],axis=0)
df.head()

In [ ]:
new_headers = []
for header in df.columns: # data.columns is your list of headers
    header = header.strip('"') # Remove the quotes off each header
    new_headers.append(header) # Save the new strings without the quotes
df.columns = new_headers # Replace the old headers with the new list

In [ ]:
df.dtypes

In [ ]:
df['concentration'] = pd.to_numeric(df['concentration'].replace("x",np.nan))

In [ ]:
# Parse String To DateTime
df['DateTime']=pd.to_datetime(df['monitordate'],
                              format='%Y-%m-%d')

In [ ]:
df.dtypes

# 呈現資料分佈的圖形制作

## 直方圖 Histogram


In [ ]:
pm25 = df[df['itemengname']=="PM2.5"]
sns.histplot(data=pm25,x="concentration")
plt.show()

In [ ]:
sns.histplot(data=df[df['itemengname']=="PM10"],x="concentration")
plt.show()

### Hands-on

看看一氧化碳(CO)的資料分布


In [ ]:
#| eval: false
#| echo: true
# 請按照題目要求修改
# 如修改正確，執行後會呈現圖形
# 修改錯誤則會有錯誤訊息
# 錯誤訊息的最下方為 KeyError: '修改這裡'，表示我們的資料表中沒有'修改這裡'這個欄位，
sns.histplot(data=df[df['itemengname']=="修改這裡"],x="concentration")
plt.show()

## 散佈圖 Scatter plot

除了PM2.5濃度的數值分佈、PM2.5濃度隨著時間的變化外，我們可能也會對各項空污數據間的關聯性有興趣，此時可以用散佈圖將有興趣的濃度分別放在x軸與y軸。在Python中，可用`seaborn`套件的`scatterplot()`函數作圖，輸入函數有x軸與y軸數據，分列如下：

x軸：PM2.5濃度 y軸：PM10濃度


In [ ]:
sns.scatterplot(df[df['itemengname']=="PM2.5"]['concentration'],df[df['itemengname']=="PM10"]['concentration'])
plt.show()

由上圖可知，PM2.5和PM10的資料分佈有一定關係，為了將此關係視覺化，可在散佈圖加上迴歸線。

## 散佈圖 Scatter plot + 迴歸線

在散佈圖中加上迴歸線可使用`seaborn`套件的`regplot()`函數，使用方法與`scatterplot()`函數相同。


In [ ]:
sns.regplot(df[df['itemengname']=="PM2.5"]['concentration'],df[df['itemengname']=="PM10"]['concentration'])
plt.show()

### Hands-on

試著看看最近的空氣污染資料中，NO2濃度與SO2濃度有沒有相關？


In [ ]:
#| eval: false
#| echo: true
# 請按照題目要求修改
# 如修改正確，會呈現圖形
# 如修改錯誤，則會有錯誤訊息
# 錯誤訊息的最下方為 KeyError: '修改這裡'，表示我們的資料表中沒有'修改這裡'這個欄位，
sns.regplot(df[修改這裡]['concentration'],df[修改這裡]['concentration'])
plt.show()

# 比較各組或不同時間數據的圖形制作

## 折線圖 Line chart

在前述章節中，我們學會如何利用視覺化圖形觀察資料分佈，但若想要比較各組或不同時間的數據，如PM2.5濃度隨時間的變化，就需要使用其他視覺化的方法。 為了呈現各項數據與時間的關係，在此一樣先前範例載入的北京空氣污染數據。 如何呈現每天 PM2.5濃度的變化？

可使用`seaborn`套件的`lineplot()`函數畫折線圖，此函數需要兩個參數，分別為x與y

-   x軸：日期
-   y軸：PM2.5濃度


In [ ]:
sns.lineplot(x=pm25[pm25['sitename']=="麥寮"]['DateTime'], y=pm25[pm25['sitename']=="麥寮"]['concentration'])
plt.show()

### Hands-on

請試著呈現林口測站在2024/1/1\~2024/1/15的PM2.5濃度起伏


In [ ]:
#| eval: false
#| echo: true
# 請按照題目要求修改
# 如修改正確，會呈現圖形
# 如修改錯誤，則會有錯誤訊息
# 錯誤訊息的最下方為 KeyError: '修改這裡'，表示我們的資料表中沒有'修改這裡'這個欄位
pm25Linkou = pm25[pm25['sitename']=="修改這裡"]
sns.lineplot(pm25Linkou['DateTime'], pm25Linkou['修改這裡'])
plt.show()

## \[進階\] 如何在折線圖中同時呈現所有氣體變化呢？

用單一折線圖有時不管用


In [ ]:
sns.lineplot(df202401[df202401['itemengname']=="PM2.5"]['DateTime'], df202401[df202401['itemengname']=="PM2.5"]['concentration'])
sns.lineplot(df202401[df202401['itemengname']=="PM10"]['DateTime'], df202401[df202401['itemengname']=="PM10"]['concentration'])
sns.lineplot(df202401[df202401['itemengname']=="CO"]['DateTime'], df202401[df202401['itemengname']=="CO"]['concentration'])
sns.lineplot(df202401[df202401['itemengname']=="NO2"]['DateTime'], df202401[df202401['itemengname']=="NO2"]['concentration'])
sns.lineplot(df202401[df202401['itemengname']=="O3"]['DateTime'], df202401[df202401['itemengname']=="O3"]['concentration'])
plt.show()

In [ ]:
g = sns.FacetGrid(df202401[df202401['itemengname'].isin("PM2.5","PM10","CO","NO2","O3")], row="itemengname",height=2, aspect=6, sharey=False)
g.map(sns.lineplot, "DateTime", "concentration")

## 管制圖 Control chart

管制圖可視為折線圖的延伸，如上圖所示，因PM2.5濃度在自然狀態下就會高低起伏，如何判斷正常的變化以及不正常的變化就變得非常重要，使用管制圖可快速判斷不正常的數值起伏。 如需了解更多管制圖的背景知識，可參考[wiki](https://zh.wikipedia.org/wiki/%E7%AE%A1%E5%88%B6%E5%9C%96)

在此使用平均值加減兩倍標準差以及加減三倍標準差當作管制界線，並使用`matplotlib`套件的`axhline()`函數將平均值與管制界線加入原有的折線圖中。因`seaborn`是基於`matplotlib`套件為基礎開發的更高階套件，因此圖形可直接疊加。 以下範例中`axhline()`函數共有以下輸入參數

-   y: 計算平均值`mean()`，計算標準差`std()`，`skipna=True`為跳過空值
-   color: 修改線條顏色 （綠色g、黃色y、紅色r），其他顏色設定可參考[文件](https://matplotlib.org/tutorials/colors/colors.html)
-   linestyle: 修改線條樣式 （虛線），其他樣式設定可參考[文件](https://matplotlib.org/3.1.0/gallery/lines_bars_and_markers/linestyles.html)


In [ ]:
sns.lineplot(dfpm25['DateTime'], dfpm25['concentration'])
plt.axhline(y=dfpm25['concentration'].mean(skipna=True), 
            color='g', linestyle='-')
plt.axhline(y=dfpm25['concentration'].mean(skipna=True)+
            2*dfpm25['concentration'].std(), 
            color='y', linestyle=':')
plt.axhline(y=dfpm25['concentration'].mean(skipna=True)-
            2*dfpm25['concentration'].std(), 
            color='y', linestyle=':')
plt.axhline(y=dfpm25['concentration'].mean(skipna=True)+
            3*dfpm25['concentration'].std(), 
            color='r', linestyle=':')
plt.axhline(y=dfpm25['concentration'].mean(skipna=True)-
            3*dfpm25['concentration'].std(), 
            color='r', linestyle=':')
plt.show()

由上圖可知，若以三倍標準差當作管制界線，一月初與一月底的幾個高峰可能為不正常變化，應考慮調查造成此變化的原因。

## 長條圖 Bar chart

若我們不是想要比較資料在不同日期的變化，而是想要比較資料**所屬類別的數量**，屬於各類別的數量比較，可用長條圖。

如比較各測站PM2.5超標的天數，可使用`seaborn`套件的`barplot()`函數，其中輸入兩個參數

-   x軸: 測站名稱
-   y軸: PM2.5超標的天數

首先使用pandas的groupby().size().reset_index()計算資料中個風向的計數資料，並儲存成wddf變數。


In [ ]:
pm25_count=pm25[pm25['concentration']>35].groupby('sitename').size().reset_index(name='size')
pm25_count.sort_values('size', ascending=False).head()

In [ ]:
sns.barplot(x=pm25_count['sitename'],y=pm25_count['size'])
plt.show()

In [ ]:
pm25_sorted = pm25_count.sort_values('size')
sns.barplot(x=pm25_sorted['sitename'],y=pm25_sorted['size'])
plt.show()

由此圖可知此資料區間中，吹NNE風向的天數最多，再來是N，以及NE。

## 盒鬚圖 Box plot

除了用直方圖得知PM2.5的濃度分佈外，如果想知道每個測站 PM2.5濃度 的分佈（中位數、四分位差、離群值等）是否有差異，可使用盒鬚圖，盒鬚圖的各點意義如下：

-   x軸：測站
-   y軸：PM2.5濃度
-   盒（方塊）中線：中位數
-   盒（方塊）上緣：第三 四分位差
-   盒（方塊）下緣：第一 四分位差
-   鬚（線條）上緣：
-   鬚（線條）下緣：
-   點：離群值

使用seaborn套件的boxplot()函數，可製作盒鬚圖，輸入參數為：

-   x軸：月份
-   y軸：各月份要比較的數據分佈（此為PM2.5）


In [ ]:
pm25[['sitename','concentration']]
sns.boxplot( x=pm25["sitename"], y=pm25["concentration"] )
plt.show()

### Hands-on

使用盒鬚圖比較各月PM2.5濃度有無差異


In [ ]:
#| eval: false
#| echo: true
# 請按照題目要求修改
# 如修改正確，會呈現一樣的圖形
# 如修改錯誤，則會有錯誤訊息
# 錯誤訊息的最下方為 KeyError: '修改這裡'，表示我們的資料表中沒有'修改這裡'這個欄位，

sns.boxplot(x=pm25["修改這裡"], y=pm25["concentration"] )
plt.show()

# 呈現相關性的圖形制作

## 泡泡圖 Bubble plot -1

泡泡圖是散佈圖的延伸，散佈圖除了可以觀察資料分佈外，也可呈現兩組資料的相關性。

在散佈圖中，每個點代表一組X與Y的數值，因此可比較XY數值，也可看出X與Y的關係。如比較PM2.5與PM10是否有關，在先前的範例中，可看出PM2.5和PM10是顯著的相關，也就是PM2.5越高，PM10就越高。

泡泡圖維持散佈圖的X與Y軸關係，但在點的大小上增加變化，將原本二維(XY)的圖形延伸成三維(XY與泡泡大小)。

## 泡泡圖 Bubble plot -2

首先如先前的教學，我們先做一般的散佈圖，X軸為PM2.5，Y軸為PM10：


In [ ]:
sns.scatterplot(df[df['itemengname']=="PM2.5"]['concentration'],df[df['itemengname']=="PM10"]['concentration'])
plt.show()

## 泡泡圖 Bubble plot -3


In [ ]:
sns.scatterplot(df[df['itemengname']=="PM2.5"]['concentration'],df[df['itemengname']=="PM10"]['concentration'],size=df[df['itemengname']=="WIND_SPEED"]['concentration'])
plt.show()

由上圖可以發現各點的大小差距不大，導致分不太出來各點的大小差距

## 泡泡圖 Bubble plot -4

可以透過sizes參數設定最大點與最小點的大小，即可畫出點大小差異較大的圖形


In [ ]:
sns.scatterplot(df[df['itemengname']=="PM2.5"]['concentration'],df[df['itemengname']=="PM10"]['concentration'],size=df[df['itemengname']=="WIND_SPEED"]['concentration'],sizes=(20, 200))
plt.show()

### Hands-on

使用泡泡圖呈現NO2與SO2的關係，並用氣溫（TEMP）當作泡泡大小，觀察這三個資料是否有相關


In [ ]:
#| eval: false
#| echo: true
# 請按照題目要求修改
# 如修改正確，會呈現一樣的圖形
# 如修改錯誤，則會有錯誤訊息
# 錯誤訊息的最下方為 KeyError: '修改這裡'，表示我們的資料表中沒有'修改這裡'這個欄位，
sns.scatterplot(df[df['itemengname']=="修改這裡"]['concentration'],df[df['itemengname']=="修改這裡"]['concentration'],size=df[df['itemengname']=="修改這裡"]['concentration'],sizes=(20, 200))
plt.show()

# 進階資料作圖

## 熱度圖 Heatmap

熱度圖使用顏色的深淺來表示數值的大小，通常會搭配XY兩軸的變量，所以使用一張圖就能表示三個維度的資訊，使用seaborn套件的heatmap()函數，即可製作熱度圖。在製作熱度圖之前，必須先將輸入資料處理成對應格式，舉例來說，若想使用熱度圖呈現每天（X軸為日期），PM2.5、PM10、SO2、NO2（Y軸為此四種污染物）的濃度（著色），我們必須將Pandas資料框的index設定為日期，PM2.5、PM10、SO2以及NO2為欄位名稱，各欄位的值是當日該物質的濃度。

最後將資料放入seaborn套件的heatmap()函數，即可製作熱度圖。 有時使用內建色盤效果不一定好，可以透過設定cmap來調整色盤。 cmap的選擇可參考[官方文件](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html)


In [ ]:
df_long = df[df['itemengname'].isin("PM2.5","PM10","CO","NO2","O3")]
df_long_linkou=df_long[df_long['sitename']=="林口"][["DateTime","itemengname","concentration"]]
df_wide_linkou = pd.pivot(df_long_linkou, index='itemengname', columns='DateTime',values='concentration')
ax = sns.heatmap(df_wide_linkou, cmap="YlGnBu")
plt.show()

## 月曆熱度圖

月曆熱度圖是熱度圖的延伸，一般來說，月曆熱度圖的各維度為：

-   X軸代表每年的"週"
-   Y軸代表星期一至星期天
-   顏色的深淺則代表欲觀察數值

通常是為了觀察或呈現某一數值是否有季節或是隨著星期變化的趨勢。 在Python中，calmap套件提供非常快速製作月曆熱度圖的方法。與製作熱度圖相同，在製作月曆熱度圖時，也必須對資料做一定的處理。 首先將日期與感興趣的資料取出，方便後續呈現


In [ ]:
import calmap
df2023=df[df["DateTime"]<'2024-01-01']
df2023=df2023.set_index(['DateTime'])
calmap.yearplot(df2023[df2023['itemengname']=='PM2.5']['concentration'], year=2023)
plt.show()

## 矩形圖 Tree map

Treemap(矩形式樹狀結構繪圖法)是以二維平面的方式展示包含階層結構（hierarchical）形式的統計資訊，可設定得值包括面積、顏色以及階層。


In [ ]:
import squarify 
df_sq = pd.DataFrame({'nb_people':[8,3,4,2], 
                   'group':["group A", "group B", "group C", "group D"] })
df_sq

## 矩形圖 Tree map


In [ ]:
squarify.plot(sizes=df_sq['nb_people'], 
              label=df_sq['group'], alpha=.8 )
plt.axis('off')
plt.show()

# 圖形樣式參數設定

https://github.com/yijutseng/DataVisPython/blob/master/DataVisCode.ipynb \# 參考資料

## Python資料視覺化參考資料

https://github.com/yijutseng/DataVisPython/blob/master/DataVisCode.ipynb